In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = os.getenv('LANGSMITH_PROJECT')
os.environ['LANGSMITH_TRACING_V2'] = os.getenv('LANGSMITH_TRACING_V2')
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

result = llm.invoke("What are deep agents?")

In [15]:
result.content

"Deep agents refer to a type of artificial intelligence (AI) that combines the concepts of deep learning and intelligent agents. In essence, deep agents are AI systems that use deep learning techniques, such as neural networks, to make decisions and take actions in complex environments.\n\nTraditional intelligent agents are software programs that perceive their environment, reason about the current state, and take actions to achieve specific goals. Deep agents take this concept a step further by leveraging deep learning algorithms to improve their decision-making and problem-solving capabilities.\n\nThe key characteristics of deep agents include:\n\n1. **Deep learning**: Deep agents use deep learning techniques, such as convolutional neural networks (CNNs), recurrent neural networks (RNNs), or transformers, to process and analyze data from their environment.\n2. **Intelligent behavior**: Deep agents exhibit intelligent behavior by making decisions, taking actions, and adapting to chang

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}")
])

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [18]:
chain = prompt | llm
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002C72C8C0FB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C72C6C3B30>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [20]:
response = chain.invoke({"input":"can you tell me the meaning of life?"})

In [21]:
response.content

'What a profound and complex question. The meaning of life is a topic that has been debated and explored by philosophers, theologians, scientists, and many others for centuries. While there\'s no one definitive answer, I can offer some insights and perspectives to consider.\n\nThe meaning of life can be understood in various ways, depending on one\'s cultural, personal, and philosophical beliefs. Here are a few possible interpretations:\n\n1. **Hedonistic perspective**: The meaning of life is to seek happiness, pleasure, and fulfillment. This view emphasizes the pursuit of personal desires, comfort, and enjoyment.\n2. **Existentialist perspective**: Life has no inherent meaning, and it\'s up to each individual to create their own purpose and significance. This view emphasizes personal freedom, choice, and responsibility.\n3. **Religious or spiritual perspective**: The meaning of life is often tied to a higher power, divine plan, or spiritual purpose. This view emphasizes the connection

In [23]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

response = chain.invoke({"input":"tell about telsa"})

print(response)

Tesla, Inc. is an American electric vehicle and clean energy company founded in 2003 by Elon Musk, Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright. The company is named after the Serbian-American inventor and engineer Nikola Tesla, who is best known for his contributions to the development of the modern alternating current (AC) electricity system.

**History:**

Tesla was initially founded as a company called "Tesla Motors" with the goal of producing high-performance, all-electric sports cars. The company's first product was the Tesla Roadster, which was launched in 2008. The Roadster was a groundbreaking vehicle that could accelerate from 0-60 mph in just 3.9 seconds and had a range of over 200 miles on a single charge.

In 2010, Tesla went public with an initial public offering (IPO) that raised $226 million. The company used this funding to develop new products, including the Model S, which was launched in 2012. The Model S was a full-size luxury sedan that could seat 

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser = JsonOutputParser()
prompt = PromptTemplate(
    template = "answer the user query \n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions":output_parser.get_format_instructions()}
)

In [5]:
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='answer the user query \n{format_instructions}\n{query}')

In [11]:
chain = prompt | llm | output_parser
reponse = chain.invoke({"query": "what is the capital of France?"})
print(reponse['answer'])

The capital of France is Paris.


In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant.\n{format_instructions}"
    ),
    ("human", "{query}")
])

# Inject format instructions properly
prompt = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

chain = prompt | llm | output_parser

result = chain.invoke({
    "query": "What is the capital of France?"
})

print(result)


{'response': 'The capital of France is Paris.'}
